# Tensorflow Softmax

## Role of placeholder

The placeholder allows you to feed data into the computation graph by way of feed dictionaries. This gives you an entry point from your non-TensorFlow code in Python so you can run a graph on new inputs.

## Automatic Differentiation

TensorFlow's automatic differentiation obviates the need for explicit gradient definition because it decomposes the graph into distinct `Op` components, each of which it has defined the gradient for. It then applies the chain rule to the graph, computes all relevant gradients, and runs a special training `Op` apply updates to the affected `Tensor` components.

# Deep NER

Named Entity Recognition involves predicting one of 4 classes (Person, Organization, Location, Miscellaneous) or a 5th null class (not a named entity) - here we are modeling it using a neural network.

## Gradient Derivations

### Base Model

#### Model Structure

The model is a 1-hidden layer neural net that classifies whether the center word is a certain class based on a small window. The windows is represented as a concatenation of 1-hot vectors that share an embedding lookup and pass through a hidden layer and softmax output layer.

$x^{(t)} = [x_{t-1} L,\ x_t L,\ x_{t+1} L] \in \mathbb R^{3d}$

For brevity, we'll just use $x$, knowing we mean $x^{(t)}$

$L \in \mathbb R^{|V| \times d}$ is an embedding matrix

$h = \tanh(x^{(t)} W + b_1)$

$\hat y = softmax(h U + b_2)$

$J(\theta) = CE(y, \hat y) = - \sum_{i = 1}^5 y_i \log \hat y_i$

The cost is the sum cross entropy cost over all 4 classes plus the 5th null class, which turns out to have the most examples.

#### Gradients from the top

Let $z_2 = hU + b_2$ and $z_1 = xW + b_1$:

Then $\frac{\partial J}{\partial z_2} = \hat y - y = \delta_2 \in \mathbb{R}^5$

$\frac{\partial J}{\partial U} = \frac{\partial z_2}{\partial U} \otimes \frac{\partial J}{\partial z_2} = h^T \delta_2 \in \mathbb{R}^{100 x 5}$

$\frac{\partial J}{\partial b_2} = \delta_2 $

$\frac{\partial J}{\partial z_1} = tanh'(z_1) \odot \frac{\partial J}{\partial h} = tanh'(z_1) \odot \delta_2 U^T = \delta_1 \in \mathbb{R}^{100}$

Let $\delta_1 = \frac{\partial J}{\partial z_1}$

$\frac{\partial J}{\partial W} = \frac{\partial z_1}{\partial W} \otimes \frac{\partial J}{\partial z_1} = x^T \delta_1 \in \mathbb{R}^{150 x 100}$

$\frac{\partial J}{\partial b_1} = \delta_1 $

$\frac{\partial J}{\partial L_i} = \frac{\partial x}{\partial L_i} \otimes \frac{\partial z_1}{\partial x} \frac {\partial J}{\partial z_1} = [x_{t-1} ,\ x_t ,\ x_{t+1} ]^T_i \delta_1 W^T$

Where 

$$\frac{\partial z_1}{\partial x} \in \mathbb{R}^{3d}$$

$$[x_{t-1} ,\ x_t ,\ x_{t+1} ]^T \delta_1 W^T \in \mathbb{R}^{|V| x 50}$$

### Regularization

#### Full cost function

$ J_{reg}(\theta) = \frac{\lambda}{2} \left[ \sum_{i, j} W_{i,j}^2 + \sum_{k, l} U_{k, l}^2 \right]$

$ J_{full}(\theta) = J(\theta) + J_{reg}(\theta) $

#### Full gradient

$\frac{\partial J_{full}(\theta)}{\partial W} = \frac{\partial J(\theta)}{\partial W} + \frac{\partial J_{reg}(\theta)}{\partial W}$

$= x^T ((\hat y - y) U^T) \odot tanh'(xW + b_1)) + \frac{\partial J_{reg}(\theta)}{\partial W}$

$= x^T ((\hat y - y) U^T) \odot tanh'(xW + b_1)) + \lambda W$

$\frac{\partial J_{full}(\theta)}{\partial U} = \frac{\partial J(\theta)}{\partial U} + \frac{\partial J_{reg}(\theta)}{\partial U}$

$= h^T (\hat y - y) + \frac{\partial J_{reg}(\theta)}{\partial U}$

$= h^T (\hat y - y) + \lambda U$